In [1]:
# import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np 
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)
import warnings
warnings.filterwarnings('ignore')

2.3.1


## 一、定义常量

In [2]:
#定义常量
t=tf.constant([[1.,2.,3.],[4.,5.,6.]])

In [3]:
#索引操作
print(t) 
print(t[:,1:]) #切片,行全要，列从第2列开始一直到最后
print(t[...,1]) #切片，行全要，列只取第2列

tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[2. 3.]
 [5. 6.]], shape=(2, 2), dtype=float32)
tf.Tensor([2. 5.], shape=(2,), dtype=float32)


In [4]:
#统计运算
print(t+10) #加法
print(tf.square(t)) #平方
print(t @ tf.transpose(t)) #矩阵的乘法

tf.Tensor(
[[11. 12. 13.]
 [14. 15. 16.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[ 1.  4.  9.]
 [16. 25. 36.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[14. 32.]
 [32. 77.]], shape=(2, 2), dtype=float32)


In [5]:
#将tensorflow转换成numpy
print(t.numpy()) #tensorflow转成numpy
print(np.square(t))
np_t=np.array([[1,2,3],[4,5,6]])
print(tf.constant(np_t)) #numpy转成tensorflow

[[1. 2. 3.]
 [4. 5. 6.]]
[[ 1.  4.  9.]
 [16. 25. 36.]]
tf.Tensor(
[[1 2 3]
 [4 5 6]], shape=(2, 3), dtype=int32)


In [6]:
#scalars
t=tf.constant(2.718)  #直接输入数字的话就是表示的是0维矩阵
print(t.numpy())
print(t.shape) #如果输出结果为()而没有内容的时候说明是0维的矩阵

2.718
()


In [7]:
#字符串
t=tf.constant('cafe')
print(t)
print(tf.strings.length(t)) #查看字符串长度，显示的是长度为4，shape=()表示的是0维矩阵
print(tf.strings.unicode_decode(t,'UTF8')) #把字符串转换成UTF8类型的数字编码

tf.Tensor(b'cafe', shape=(), dtype=string)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor([ 99  97 102 101], shape=(4,), dtype=int32)


In [8]:
#数组字符串
t=tf.constant(['cafe','coffee','咖啡'])
print(tf.strings.length(t))
print(tf.strings.unicode_decode(t,'UTF8'))

tf.Tensor([4 6 6], shape=(3,), dtype=int32)
<tf.RaggedTensor [[99, 97, 102, 101], [99, 111, 102, 102, 101, 101], [21654, 21857]]>


In [9]:
#ragged tensor将字符串转换成不规则长度
r=tf.ragged.constant([[11,12],[21,22,23],[],[41]])
print(r)
print(r[1])
print(r[1:2])

<tf.RaggedTensor [[11, 12], [21, 22, 23], [], [41]]>
tf.Tensor([21 22 23], shape=(3,), dtype=int32)
<tf.RaggedTensor [[21, 22, 23]]>


In [10]:
#常量拼接
r2=tf.ragged.constant([[51,52],[],[71]])
print(tf.concat([r,r2],axis=0)) #行拼接
r3=tf.ragged.constant([[53,54],[],[72]])
print(tf.concat([r2,r3],axis=1))

<tf.RaggedTensor [[11, 12], [21, 22, 23], [], [41], [51, 52], [], [71]]>
<tf.RaggedTensor [[51, 52, 53, 54], [], [71, 72]]>


In [11]:
#ragged 转换为普通的tensor
print(r.to_tensor())

tf.Tensor(
[[11 12  0]
 [21 22 23]
 [ 0  0  0]
 [41  0  0]], shape=(4, 3), dtype=int32)


In [12]:
#sparse tensor
s=tf.SparseTensor(indices=[[0,1],[1,0],[2,3]],values=[1,2,3],dense_shape=[3,4])
print(s)
print(tf.sparse.to_dense(s)) #转换成sparse的密集举证

SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 0]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([1 2 3], shape=(3,), dtype=int32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
tf.Tensor(
[[0 1 0 0]
 [2 0 0 0]
 [0 0 0 3]], shape=(3, 4), dtype=int32)


## 二、定义变量

In [13]:
#变量的定义
v=tf.Variable([[1,2,3],[4,5,6]])
print(v)
print(v.value())
print(v.numpy())

<tf.Variable 'Variable:0' shape=(2, 3) dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]])>
tf.Tensor(
[[1 2 3]
 [4 5 6]], shape=(2, 3), dtype=int32)
[[1 2 3]
 [4 5 6]]


In [14]:
#给变量重新赋值
v.assign(2*v)
v[0,1].assign(42) #把第1行的第2个数字变成42
v[1].assign([7,8,9])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=int32, numpy=
array([[ 2, 42,  6],
       [ 7,  8,  9]])>

## 三、tf.function
都是函数，但是tf.function（）转换后的函数运算更快

In [15]:
#tf.function的作用是把普通的代码块变成tensorflow里面的图

In [16]:
#python普通的函数
def scalerd_elu(z,scale=1.0,alpha=1.0):
    # if z>0 then scale *z else scale*alpha*tf.nn.elu(z)
    is_positive=tf.greater_equal(z,0.0)
    return scale*tf.where(is_positive,z,alpha*tf.nn.elu(z))
print(scalerd_elu(tf.constant(-3.,)))
print(scalerd_elu(tf.constant([-3.,-2.5])))

tf.Tensor(-0.95021296, shape=(), dtype=float32)
tf.Tensor([-0.95021296 -0.917915  ], shape=(2,), dtype=float32)


In [17]:
#将python普通的函数转换成tf.function（）类型的图
scaled_elu_tf=tf.function(scalerd_elu)
print(scaled_elu_tf(tf.constant(-3.)))

tf.Tensor(-0.95021296, shape=(), dtype=float32)


In [18]:
#也可以使用魔法糖方式直接装饰普通的python方法
@tf.function
def scalerd_elu(z,scale=1.0,alpha=1.0):
    # if z>0 then scale *z else scale*alpha*tf.nn.elu(z)
    is_positive=tf.greater_equal(z,0.0)
    return scale*tf.where(is_positive,z,alpha*tf.nn.elu(z))
print(scalerd_elu(tf.constant(-3.,)))
print(scalerd_elu(tf.constant([-3.,-2.5])))

tf.Tensor(-0.95021296, shape=(), dtype=float32)
tf.Tensor([-0.95021296 -0.917915  ], shape=(2,), dtype=float32)
